<a href="https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/beta/ESMFold_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**ESMFold API**
for more details see: [Github](https://github.com/facebookresearch/esm/tree/main/esm), [Preprint](https://www.biorxiv.org/content/10.1101/2022.07.20.500902v1)

#### **Tips and Instructions**
- click the little ▶ play icon to the left of each cell below.

#### **Limitations**
- max total length=400
- for lengths > 400, try our [colab notebook](https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/ESMFold.ipynb)
---

In [ ]:
#@title run **ESMFold**
import os, time, re, requests
if not os.path.isfile("colabfold.py"):
  os.system("pip -q install pyppeteer nest_asyncio")
  import asyncio
  import nest_asyncio
  from pyppeteer import launch
  import base64

  # Apply nest_asyncio to enable nested event loops
  nest_asyncio.apply()

  async def fetch_blob_content(page, blob_url):
    blob_to_base64 = """
    async (blobUrl) => {
        const blob = await fetch(blobUrl).then(r => r.blob());
        return new Promise((resolve) => {
            const reader = new FileReader();
            reader.onloadend = () => resolve(reader.result);
            reader.readAsDataURL(blob);
        });
    }
    """
    base64_data = await page.evaluate(blob_to_base64, blob_url)
    _, encoded = base64_data.split(',', 1)
    return base64.b64decode(encoded)

  async def extract_pdb_file_download_link_and_content(url):
    browser = await launch(headless=True, args=['--no-sandbox', '--disable-setuid-sandbox'])
    page = await browser.newPage()
    await page.goto(url, {'waitUntil': 'networkidle0'})
    elements = await page.querySelectorAll('a.btn.bg-purple')
    for element in elements:
        href = await page.evaluate('(element) => element.getAttribute("href")', element)
        if 'blob:https://esmatlas.com/' in href:
            content = await fetch_blob_content(page, href)
            await browser.close()
            return href, content
    await browser.close()
    return "No PDB file link found.", None

  def esmfold_api(sequence):
    url = f'https://esmatlas.com/resources/fold/result?fasta_header=%3Eunnamed&sequence={sequence}'
    result = asyncio.get_event_loop().run_until_complete(extract_pdb_file_download_link_and_content(url))
    if result[1]:
      pdb_str = result[1].decode('utf-8')
      return pdb_str
    else:
      return "Failed to retrieve PDB content."

  os.system("pip -q install py3Dmol")
  os.system("wget -qnc https://raw.githubusercontent.com/sokrypton/ColabFold/main/beta/colabfold.py")
  os.system("mkdir -p tmp")

import colabfold as cf


sequence = "GWSTELEKHREELKEFLKKEGITNVEIRIDNGRLEVRVEGGTERLKRFLEELRQKLEKKGYTVDIKIE" #@param {type:"string"}
sequence = re.sub("[^A-Z]", "", sequence.upper())
#assert len(sequence) <= 400, "error: max length supported is 400"
hash = cf.get_hash(sequence)[:5]
pdb_filename = f"tmp/prediction_{hash}.pdb"
if not os.path.isfile(pdb_filename):
  pdb_str = esmfold_api(sequence)
  with open(pdb_filename,"w") as out:
    out.write(pdb_str)

color = "confidence" #@param ["confidence", "rainbow"]
if color == "confidence": color = "lDDT"
show_sidechains = False #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}
v = cf.show_pdb(pdb_filename,
                show_sidechains,
                show_mainchains, color,
                color_HP=True,
                size=(800,480),
                vmin=0.5, vmax=0.9)
v.setHoverable({}, True,
               '''function(atom,viewer,event,container){if(!atom.label){atom.label=viewer.addLabel("      "+atom.resn+":"+atom.resi,{position:atom,backgroundColor:'mintcream',fontColor:'black'});}}''',
               '''function(atom,viewer){if(atom.label){viewer.removeLabel(atom.label);delete atom.label;}}''')
v.show()
if color == "lDDT":
  cf.plot_plddt_legend().show()

In [ ]:
#@title download prediction
from google.colab import files
files.download(pdb_filename)